In [ ]:
import cv2
import numpy as np

# --- 【穩定性修改 1】 ---
# 建立全域變數 (記憶體)，用來儲存上一幀的車道線
last_known_left_line = None
last_known_right_line = None
# -------------------------

# --- 步驟 5 & 8 的輔助函式 ---

def region_of_interest(image):
    """
    (步驟 5 輔助)
    建立一個梯形遮罩 (ROI)，只保留我們感興趣的道路區域。
    """
    height = image.shape[0]
    width = image.shape[1]
    
    # 這些頂點需要根據您的影片畫面和相機角度來調整
    # 目前設定為一個通用的梯形
    polygons = np.array([
        [(int(width*0.18), height),          # 1. 左下 (從 0.1 變 0.2，更靠內)
         (int(width*0.48), int(height*0.73)), # 2. 左上 (從 0.45 變 0.48，更靠內；從 0.6 變 0.65，更低)
         (int(width*0.5), int(height*0.73)), # 3. 右上 (從 0.55 變 0.52，更靠內；從 0.6 變 0.65，更低)
         (int(width*0.7), height)]          # 4. 右下 (從 0.95 變 0.8，更靠內)
    ], dtype=np.int32)
    
    mask = np.zeros_like(image) # 建立一個全黑的遮罩
    cv2.fillPoly(mask, polygons, 255) # 將多邊形區域填滿白色
    
    # 執行 AND 運算，只保留遮罩區域內的邊緣
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image, polygons

def make_coordinates(image, line_parameters):
    """
    (步驟 8 輔助)
    將斜率和截距轉換回影像中的 x, y 座標。
    """
    slope, intercept = line_parameters
    y1 = image.shape[0]      # 影像底部
    y2 = int(y1 * 0.75)       # 影像的中上方 (配合 ROI)
    
    # 從 y = mx + b 推導 x = (y - b) / m
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    """
    (步驟 8 輔助)
    將 Hough 轉換偵測到的多條線段，平均成左、右兩條車道線。
    """
    left_fit = []  # 儲存左車道線的 (斜率, 截距)
    right_fit = [] # 儲存右車道線的 (斜率, 截距)
    
    if lines is None:
        return None, None
        
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        
        # 忽略垂直線 (斜率無限大)
        if x1 == x2:
            continue
            
        # 計算斜率和截距
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        
        # 根據斜率過濾
        # 在影像座標中，左車道線斜率為負，右車道線斜率為正
        if slope < -0.3: # 負斜率 (左側)
            left_fit.append((slope, intercept))
        elif slope > 0.3: # 正斜率 (右側)
            right_fit.append((slope, intercept))
            
    # 平均左側車道線
    left_line = None
    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
        left_line = make_coordinates(image, left_fit_average)
        
    # 平均右側車道線
    right_line = None
    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        right_line = make_coordinates(image, right_fit_average)
        
    return left_line, right_line

def calculate_and_draw_deviation(image, left_line, right_line, roi_polygon_overlay):
    """
    (步驟 8 輔助)
    計算並畫出偏離中線的指示。
    """
    height, width, _ = image.shape
    
    # 1. 找到車道線在影像底部的 x 座標
    left_x_bottom = left_line[0]
    right_x_bottom = right_line[0]
    
    # 2. 計算車道中心點
    lane_center_x = (left_x_bottom + right_x_bottom) / 2
    
    # 3. 獲取影像中心點 (假設車輛攝影機在正中央)
    image_center_x = width / 2
    
    # 4. 計算偏移量 (像素)
    offset_pixels = image_center_x - lane_center_x
    
    # (可選) 轉換為公尺 (需要真實世界的校準)
    # 假設美國車道寬 3.7 公尺
    lane_width_pixels = right_x_bottom - left_x_bottom
    if lane_width_pixels != 0:
        xm_per_pix = 3.7 / lane_width_pixels
        offset_meters = offset_pixels * xm_per_pix
        direction = "left" if offset_meters > 0 else "right"
        text = f"Offset: {abs(offset_meters):.2f}m to the {direction}"
    else:
        text = "Calculating..."
        
    # 5. 在畫面上顯示文字
    cv2.putText(image, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    # 6. 畫出中線指示 (如您範例中的紅線)
    # 車道中心 (綠線)
    cv2.line(roi_polygon_overlay, (int(lane_center_x), height - 10), (int(lane_center_x), height - 40), (0, 255, 0), 3)
    
    return image, roi_polygon_overlay

def draw_lane(original_image, processed_image, left_line, right_line, roi_pts):
    """
    (步驟 8 輔助)
    將所有元素畫到最終的影像上。
    """
    # 1. 建立一個空白影像來繪製車道區域
    overlay = np.zeros_like(original_image)
    
    # 2. 畫出綠色車道區域 (如範例圖)
    if left_line is not None and right_line is not None:
        # 獲取車道線的4個頂點
        pts = np.array([[left_line[0], left_line[1]], [left_line[2], left_line[3]], 
                        [right_line[2], right_line[3]], [right_line[0], right_line[1]]], np.int32)
        cv2.fillPoly(overlay, [pts], (0, 255, 0)) # 綠色

        # --- 【請求 1: 新增動態中心線】 ---
        # 計算中心線的底部和頂部 x, y 座標
        # (// 2 是整數除法)
        center_x_bottom = (left_line[0] + right_line[0]) // 2
        center_y_bottom = left_line[1] # 影像底部 y

        center_x_top = (left_line[2] + right_line[2]) // 2
        center_y_top = left_line[3] # 影像中上方 y (來自 make_coordinates)

        # 畫一條黃色實線作為中心線
        cv2.line(overlay, (center_x_bottom, center_y_bottom), (center_x_top, center_y_top), (0, 255, 255), 2) # 黃色, 2px
        # --- 【新增結束】 ---

        # 3. 計算並畫出偏離指示
        original_image, overlay = calculate_and_draw_deviation(original_image, left_line, right_line, overlay)

    # 4. 將綠色區域與原圖合併 (半透明效果)
    # 這裡的 0.3 和 0.7 是權重，可以調整透明度
    final_image = cv2.addWeighted(original_image, 0.7, overlay, 0.4, 0)
    
    # 5. 畫出最終的(紅/藍)車道線 (可選，這裡用紅色)
    if left_line is not None:
        cv2.line(final_image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 0, 255), 5)
    if right_line is not None:
        cv2.line(final_image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 0, 255), 5)

    return final_image

# --- 主處理流程 ---

def process_frame(frame):
    """
    套用您指定的 8 步驟完整影像處理管線
    """

    global last_known_left_line
    global last_known_right_line

    # 複製原始幀
    original_frame = np.copy(frame)
    
    # === 步驟 1: 每幀圖片轉成灰階 ===
    # 注意：cv2.VideoCapture 讀取的是 BGR，不是 RGBA
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # === 步驟 2: 執行影像擴張 (Dilate) ===
    # 這個步驟在模糊 *之前* 執行比較少見，但照您的要求實作
    # 它會強化影像中的亮點 (例如白色車道線)
    kernel_dilate = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(gray, kernel_dilate, iterations=1)

    # === 步驟 3: Apply Gaussian blur ===
    # 使用 5x5 的核來平滑影像，減少雜訊
    blur = cv2.GaussianBlur(dilated, (5, 5), 0)

    # === 步驟 4: 執行 Canny Edge detection ===
    # 50 和 150 是常用的低/高閾值
    canny = cv2.Canny(blur, 50, 150)
    
   
    
    # === 步驟 5: 影像遮罩 (ROI) ===
    # 遮罩 Canny 的結果，只保留道路區域的邊緣
    masked_canny, roi_polygon_pts = region_of_interest(canny)

    # === 步驟 6: 執行影像侵蝕 (Erode) ===
    # 這個步驟在 Canny *之後* 執行，有助於去除細小的雜訊邊緣
    #kernel_erode = np.ones((3, 3), np.uint8)
    #eroded = cv2.erode(masked_canny, kernel_erode, iterations=1)
    #return masked_canny
    # === 步驟 7: 執行 Hough Transformation ===
    # 從邊緣影像中找出所有可能的線段
    # 參數：(影像, rho精度, theta精度, 閾值, 最小線長, 最大線間距)
    lines = cv2.HoughLinesP(masked_canny, 
                            rho=2, 
                            theta=np.pi/180, 
                            threshold=100, 
                            minLineLength=40, 
                            maxLineGap=5)

    # === 步驟 8: 自己寫車道線辨識，並畫圖 ===
    # 1. 平均/擬合線段
    left_line, right_line = average_slope_intercept(frame, lines)
    
    # --- 【穩定性修改 3：核心邏輯】 ---
    
    # 檢查 左線
    if left_line is None:
        # 這一幀沒找到？沒關係，使用上一幀的記憶
        left_line = last_known_left_line
    else:
        # 這一幀找到了！更新記憶
        last_known_left_line = left_line

    # 檢查 右線
    if right_line is None:
        # 這一幀沒找到？使用上一幀的記憶
        right_line = last_known_right_line
    else:
        # 這一幀找到了！更新記憶
        last_known_right_line = right_line
        
    # --- 【修改結束】 ---
    
    # 2. 將線條、區域和偏移量畫回原始影像
    final_image = draw_lane(original_frame, masked_canny, left_line, right_line, roi_polygon_pts)
    
    return final_image
    #return masked_canny
    #return eroded
# --- 主程式進入點 ---

# *** 請將 'your_video.mp4' 換成您自己的道路駕駛影片檔案路徑 ***
video_path = 'C:/Users/Cho_lab/Desktop/YX/ml/LaneVideo.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: 無法開啟影片檔案 {video_path}")
    exit()

# 1. 取得原始影片的屬性
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 2. 定義輸出影片的路徑和編碼器
#    *** 您可以修改 'output_lane_video.mp4' 為您想要的路徑 ***
output_path = 'lane_detection_output.mp4'
#    使用 MP4V 編碼器 (適用於 .mp4 格式)
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

# 3. 建立 VideoWriter 物件
#    注意：(frame_width, frame_height) 必須與您寫入的影格 (processed_image) 尺寸相符
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

print(f"正在處理影片... 將儲存至 {output_path}")

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("影片播放完畢或讀取錯誤。")
        break
        
    # 將每一幀影像丟入我們的處理流程
    processed_image = process_frame(frame)
    
    # 顯示結果
    cv2.imshow('Lane Detection (Press Q to quit)', processed_image)
    
    # 按 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()

影片播放完畢或讀取錯誤。
